## The following analysis is for the CASA0006 final assignment

In this code we explore the following question: *Can unsupervised clustering reveal distinct patterns of household energy consumption across different regions of the UK, and what factors contribute to these patterns?*